# **Extração de dados da Planilha**

**Autor:** Davi Vago Soares  
**Data:** 25-08-2024


In [1]:
#Bibliotecas utilizadas

import pandas as pd
import numpy as np
import pyodbc
import warnings
warnings.filterwarnings('ignore')

In [2]:
#definindo local do arquivo utilizado

caminho = r"C:\Users\davis\OneDrive\Repositórios\Moer\moer\database-archives\registro-extracoes.xlsx"

df = pd.read_excel(caminho)

In [21]:
#Conectando ao banco de dados

try:
    server = 'localhost\\SQLEXPRESS'
    database = 'coffee'
    conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
    print("Conexão bem-sucedida!")
except pyodbc.Error as ex:
    sqlstate = ex.args[1]
    print(f"Erro de Conexão: {sqlstate}")

Conexão bem-sucedida!


In [22]:
#buscando maior id no banco para atualização incremental

# Define o comando SQL
sql_comando = """
SELECT MAX(id_extracao_pk) AS max_id
FROM [coffee].[dbo].[fato_extracoes]
"""

# Executa a consulta e cria um DataFrame
maiorId = pd.read_sql_query(sql_comando, conn)
maiorId

# Obtém o valor de max_id
valor_max_id = maiorId['max_id'].iloc[0]
valor_max_id


15

In [23]:
# Filtra o DataFrame df onde id_extracao_pk é maior que o valor de max_id
df_filtrado = df[df['id_extracao_pk'] > valor_max_id]

In [24]:
#Selecionando colunas

df = df_filtrado[['id_moedor_fk', 'id_metodo_fk',
       'id_cafe_fk', 'vl_peso_cafe', 'vl_nota', 'vl_pesoagua',
       'vl_temperatura', 'vl_regulagem_moedor', 'ds_sensorial', 'ds_corpo',
       'dt_dia_extracao', 'ts_tempo_extracao']].copy()

df

,id_moedor_fk,id_metodo_fk,id_cafe_fk,vl_peso_cafe,vl_nota,vl_pesoagua,vl_temperatura,vl_regulagem_moedor,ds_sensorial,ds_corpo,dt_dia_extracao,ts_tempo_extracao
4,1,3,1.0,20.5,4.0,300.0,93.0,110.0,Fruatas Amarelas,macio,2024-08-31,00:02:50.0000000


In [25]:
# Escrevendo dados no banco

cursor = conn.cursor()

sql_comando = """INSERT INTO [coffee].[dbo].[fato_extracoes] (
       [id_moedor_fk]
      ,[id_metodo_fk]
      ,[id_cafe_fk]
      ,[vl_peso_cafe]
      ,[vl_nota]
      ,[vl_pesoagua]
      ,[vl_temperatura]
      ,[vl_regulagem_moedor]
      ,[ds_sensorial]
      ,[ds_corpo]
      ,[dt_dia_extracao]
      ,[ts_tempo_extracao]) 
      VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

# Iterando pelas linhas do DataFrame
for array in df.values:
    # Executa o comando SQL substituindo os placeholders pelos valores da linha
    cursor.execute(sql_comando, tuple(array))

# Confirma as mudanças no banco de dados
conn.commit()